<a href="https://colab.research.google.com/github/MVFran/PyTorchForDeepLearning/blob/master/RedNeuronalConvolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hola
